In [1]:
!pip install git+https://github.com/huggingface/peft.git

  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-wmo3se4r
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-wmo3se4r
  Resolved https://github.com/huggingface/peft.git to commit 042123465ce80b420698345073ab4f38b533ea73
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.

In [2]:
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers==4.38.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.8/103.8 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible

In [3]:
import os
import transformers
import torch
from google.colab import userdata
from datasets import load_dataset
from trl import SFTTrainer
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig, GemmaTokenizer
from transformers import Trainer, AutoModelForCausalLM, AutoTokenizer
from peft import get_peft_model, LoraConfig, TaskType
from datasets import load_dataset

In [4]:
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

In [5]:
model_id = "google/gemma-2b-it"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_id, token=os.environ['HF_TOKEN'])
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map={"":0},
                                             token=os.environ['HF_TOKEN'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [7]:
os.environ["WANDB_DISABLED"] = "false"

In [10]:
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=[
 'down_proj',
 'o_proj',
 'k_proj',
 'q_proj',
 'gate_proj',
 'up_proj',
 'v_proj']
)
model = get_peft_model(model, lora_config)

In [11]:
import pandas as pd
from datasets import Dataset

# Load the dataset from the uploaded file
data = pd.read_csv('aimldst.csv')

# Convert the pandas DataFrame into a Hugging Face Dataset
data = Dataset.from_pandas(data)


In [13]:
def formatting_func(example):
    text = f"Question: {example['Question']}\nAnswer: {example['Answer']}"
    return {"formatted_text": text}

# Apply formatting
formatted_dataset = tokenized_dataset.map(formatting_func)

# Check if formatting is correct
print(formatted_dataset[0]['formatted_text'])  # Ensure this is a string


Map:   0%|          | 0/52 [00:00<?, ? examples/s]

Question: What Are the Different Types of Machine Learning?
Answer: There are three types of machine learning: Supervised Learning, Unsupervised Learning, and Reinforcement Learning.


In [16]:
print(formatted_dataset[0])


{'Question': 'What Are the Different Types of Machine Learning?', 'Answer': 'There are three types of machine learning: Supervised Learning, Unsupervised Learning, and Reinforcement Learning.', 'input_ids': [2, 1841, 5881, 573, 28398, 23141, 576, 13403, 14715, 235336], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'formatted_text': 'Question: What Are the Different Types of Machine Learning?\nAnswer: There are three types of machine learning: Supervised Learning, Unsupervised Learning, and Reinforcement Learning.'}


In [17]:

# Extract the formatted text into a list
formatted_texts = [example['formatted_text'] for example in formatted_dataset]

# Tokenize the list of formatted texts
tokenized_dataset = tokenizer(
    formatted_texts,  # Pass the list of strings
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors='pt'  # Optional: Return PyTorch tensors
)

In [18]:
formatted_texts

['Question: What Are the Different Types of Machine Learning?\nAnswer: There are three types of machine learning: Supervised Learning, Unsupervised Learning, and Reinforcement Learning.',
 'Question: What is Overfitting, and How Can You Avoid It?\nAnswer: Overfitting occurs when a model learns the training set too well, capturing noise instead of the underlying pattern. To avoid it, you can use techniques like regularization, simplifying the model, and cross-validation.',
 'Question: What is ‘training Set’ and ‘test Set’ in a Machine Learning Model?\nAnswer: The training set is the portion of the data used to train the model, while the test set is used to evaluate its performance. A common split is 70% training and 30% testing.',
 'Question: How Do You Handle Missing or Corrupted Data in a Dataset?\nAnswer: You can handle missing data by dropping rows/columns or replacing them with placeholder values using methods like isNull(), dropna(), and fillna() in Pandas.',
 'Question: How Can Y

In [40]:
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM, AutoTokenizer

def tokenize_function(examples):
    return tokenizer(
        examples['Question'],
        padding='max_length',
        truncation=True,
        max_length=512
    )

tokenized_dataset = data.map(tokenize_function, batched=True)


def add_labels(examples):
    inputs = tokenizer(
        examples['Question'],
        padding='max_length',
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )
    labels = tokenizer(
        examples['Answer'],
        padding='max_length',
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )["input_ids"]
    inputs["labels"] = labels
    return inputs

tokenized_dataset = tokenized_dataset.map(add_labels, batched=True)

training_args = TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    warmup_steps=2,
    max_steps=200,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=1,
    output_dir="outputs",
    optim="paged_adamw_8bit"
)

# Define custom Trainer to compute loss
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        # Ensure inputs are on the correct device
        labels = inputs.pop("labels").to(model.device)
        inputs = {k: v.to(model.device) for k, v in inputs.items()}

        # Forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")

        # Compute the loss
        loss_fn = torch.nn.CrossEntropyLoss()
        loss = loss_fn(logits.view(-1, logits.size(-1)), labels.view(-1))

        return (loss, outputs) if return_outputs else loss

# Initialize the Trainer
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    # eval_dataset=tokenized_dataset['validation'],  # Optional, if you have a validation set
)




Map:   0%|          | 0/52 [00:00<?, ? examples/s]

Map:   0%|          | 0/52 [00:00<?, ? examples/s]

In [41]:
trainer.train()

Step,Training Loss
1,4.841900
2,4.872100
3,10.675200
4,10.618300
5,10.594100
6,10.594000
7,11.034800
8,6.128800
9,5.878200
10,5.497100


TrainOutput(global_step=200, training_loss=4.9175390458107, metrics={'train_runtime': 492.8904, 'train_samples_per_second': 1.623, 'train_steps_per_second': 0.406, 'total_flos': 4894777933824000.0, 'train_loss': 4.9175390458107, 'epoch': 15.38})

In [44]:
prompt = "What Are the Different Types of Machine Learning?"

input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)

# Generate output
output = model.generate(input_ids, max_length=50, num_return_sequences=1)

# Decode the generated text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print(generated_text)


What Are the Different Types of Machine Learning?...

1. Supervised learning
2. Unsupervised learning
3. Reinforcement learning
4. Semi-supervised learning

Explain each type in more detail...

**


In [45]:
prompt = "What is ‘training Set’ and ‘test Set’ in a Machine Learning Model?"

input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)

# Generate output
output = model.generate(input_ids, max_length=50, num_return_sequences=1)

# Decode the generated text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print(generated_text)


What is ‘training Set’ and ‘test Set’ in a Machine Learning Model?... 

Training set is typically 80-90% of the total data, while the test set is 20-30


In [47]:
prompt = "What is Overfitting, and How Can You Avoid It?"

input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)

# Generate output
output = model.generate(input_ids, max_length=150, num_return_sequences=1)

# Decode the generated text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print(generated_text)


What is Overfitting, and How Can You Avoid It?.... 
 
. 

Overfitting is a pattern that is too closely tailored to the training data and does not generalize well to new data... 

To avoid overfitting, we can use early stopping, regularization, and reducing the number of model parameters... 

- **Early stopping** is a technique where the training data is split into training and validation sets. The model is trained on the training set and evaluated on the validation set. If the validation performance does not improve for a set number of iterations, the training process is stopped and the model is saved. This prevents the model from overfitting to the training data.. 


In [48]:
# Assuming you have fine-tuned your model and tokenizer
save_directory = "./fine_tuned_model"

# Save the model
model.save_pretrained(save_directory)

# Save the tokenizer
tokenizer.save_pretrained(save_directory)

print(f"Model and tokenizer saved to {save_directory}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Model and tokenizer saved to ./fine_tuned_model


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the model
model = AutoModelForCausalLM.from_pretrained(save_directory)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(save_directory)

# Now you can use the model and tokenizer as needed


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]